**Importe de librerías y drivers**

Las librería utilizadas para el tratamiento de datos y la creación de la base son **pandas** y **sqlalchemy** respectivamente.

Es importante destacar que la instalación de los drivers está correta cuando dentro de todos los paquetes instalados se encuentra el siguiente:

<i>'Teradata Database ODBC Driver 20.00'</i>

Si se desea cambiar la plataforma para operar las bases de datos, se debe cuidar tener la información correspondiente.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import pyodbc
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'Teradata Database ODBC Driver 20.00']

**Licencias y autentificación**

Se definen las credenciales del usuario para acceder a la base de datos de interes. Una pequeña descripción de los datos son:

<ul>
<li>teradata_driver = Driver utilizado con su correspondiente versión (En caso de cuando se crea el documento: 'Teradata Database ODBC Driver 20.00')</li>
<li>teradata_host_name = Host dipuesto por Entel para ingresar a las bases de datos, puede ser de dominio entel o una dirección IP (linkeada a las BBDD)</li>
<li>teradata_user = Usuario teradata</li> 
<li>teradata_password = Contraseña teradata</li>
<li>teradata_authentication = Autentificación de teradata</li>
<ul>
<br>




In [3]:
#Credenciales
teradata_driver = 'Teradata Database ODBC Driver 20.00'
teradata_host_name = '10.52.136.236' # puede ser la direccion IP
teradata_user = 'P_CFAUNDEZ'
teradata_password = 'Carlos_Entel_5052'

# Autentificación de teradata
teradata_authentication = 'TD2'

**Comprobando la conexión a la base**

In [4]:
#Conectado a la base solicitada
print('Connecting to DB')
cnxn = pyodbc.connect("DRIVER={%s};DBCNAME=%s;UID=%s;PWD=%s;authentication=%s"\
                      %(teradata_driver,teradata_host_name,teradata_user,teradata_password,teradata_authentication))

print('Connection achieved')

Connecting to DB
Connection achieved


**Conseguir ruta para el archivo y preprocesamiento**


Se consigue el archivo de la base deseada y se procesan los datos, eliminando los valores no deseados (NaN, registros incompletos,etc.) 

Para normalizar los meses se define un índice temporal con la operación indice_temporal = años * 12 + mes, por ejemplo: el 11-2024 tiene como codigo 2024 * 12 + 11 = 24299. Siempre será un número entero por restricción de tipo (int).

In [ ]:
def cargar_datos(ruta_archivo):
    
    #Leer el archivo en formato csv, precaución al separador
    df = pd.read_csv(ruta_archivo, sep=';')
    #Elminar data que contenga NaN en su registros
    df = df.dropna()
    #Orden cronologico
    df = df.sort_values(by=['POP','YEAR_ID','MONTH_ID']).reset_index(drop=True)
    #Definicion de un indice temporal que opera con año*12+mes, el 11-2024 tiene como codigo: 2024 * 12 + 11 = 24299 (siempre será un numero entero)
    df['TEMP_IND'] = (df['YEAR_ID'] * 12 + df['MONTH_ID']).astype(int)
    
    return df


#####

def cargar_datos_teradata(conexion_teradata, nombre_tabla):
    
    #Query para rescatar la tabla que se quiera analizar
    query = f"SELECt * FROM {nombre_tabla}"

    #Conectar a teradata y crear el dataframe con query
    with conexion_teradata.connect() as conn:
        df = pd.read_sql(query,conn)

    #Orden cronologico
    df = df.sort_values(by=['POP','YEAR_ID','MONTH_ID']).reset_index(drop=True)    

    #Elminar data que contenga NaN en su registros
    df = df.dropna()

    #Definicion de un indice temporal que opera con año*12+mes, el 11-2024 tiene como codigo: 2024 * 12 + 11 = 24299 (siempre será un numero entero)
    df['TEMP_IND'] = (df['YEAR_ID'] * 12 + df['MONTH_ID']).astype(int)
    
    return df



**Identificación de eventos**


Función principal que detecta cuando un pop entra y sale de la base de datos. El proceso tiene las siguientes interpretaciones:

<ol>
<li>Cada evento de entrada o salida es un diccionario que contiene: nombre del evento("entrada" o "salida"), POP, año y mes
<li>El primer registro siempre se toma como una entrada
<li>El último mes solo registra las entradas, no considera como si saliera ningún POP
<li>El mes de salida se toma como el último que existe registro de actividad del POP, por ejemplo, si el POP está en la base los meses 9,10 y 11 del 2024 pero desaparece el 12/2024, su mes de salida es Noviembre del respectivo año.
</ol>


In [ ]:
def identificar_eventos(df):

    #Lista vacía para almacenar eventos. Cada evento es un diccionario
    eventos = []

    ultimo_mes = df['TEMP_IND'].maX()

    #Agrupar el Dataframe por grupo de POPs
    for POP, grupo in df.groupby('POP'):
        grupo = grupo.reset_index(drop=True)  # Reiniciar índices dentro del grupo

        # Analizar cada fila del grupo
        for i in range(len(grupo)):
            # Detectar entrada al inicio o después de observar falta de información el meses contiguos
            if i == 0 or grupo.loc[i, 'TEMP_IND'] - grupo.loc[i - 1, 'TEMP_IND'] > 1:
                evento_entrada = {
                    'YEAR_ID': grupo.loc[i, 'YEAR_ID'],
                    'MONTH_ID': grupo.loc[i, 'MONTH_ID'],
                    'POP': POP,
                    'EVENTO': 'ENTRADA'
                }
                #Añadir solo si es un evento de entrada nuevo (Evitar eventos duplicados)
                if evento_entrada not in eventos:
                    eventos.append(evento_entrada)

            # Detectar salida al final o antes de una brecha 
            if (grupo.loc[i,'TEMP_IND'] != ultimo_mes and (i == len(grupo) - 1 or grupo.loc[i + 1, 'TEMP_IND'] - grupo.loc[i, 'TEMP_IND'] > 1)):
                evento_salida = {
                    'YEAR_ID': grupo.loc[i, 'YEAR_ID'],
                    'MONTH_ID': grupo.loc[i, 'MONTH_ID'],
                    'POP': POP,
                    'EVENTO': 'SALIDA'
                }
                #Añadir solo si es un evento de salida nuevo (Evitar eventos duplicados)
                if evento_salida not in eventos:
                    eventos.append(evento_salida)

    # Convertir lista de eventos en un DataFrame y eliminar duplicados (precaución extra)
    bitacora = pd.DataFrame(eventos).drop_duplicates().sort_values(by=['POP', 'YEAR_ID', 'MONTH_ID']).reset_index(drop=True)

    return bitacora

**Función de conteo por eventos**



In [ ]:
def contar_eventos(bitacora):

    # Agrupar por año, mes y tipo de evento, y contar las ocurrencias
    conteo = bitacora.groupby(['YEAR_ID', 'MONTH_ID', 'EVENTO']).size().reset_index(name='CONTEO')
    
    # Pivotar los datos para tener columnas separadas para ENTRADAS y SALIDAS
    pivot = conteo.pivot(index=['YEAR_ID', 'MONTH_ID'], columns='EVENTO', values='CONTEO').fillna(0).reset_index()
    
    # Renombrar columnas para mayor claridad
    pivot = pivot.rename(columns={'ENTRADA': 'NUM_ENTRADAS', 'SALIDA': 'NUM_SALIDAS'})
    
    return pivot

**Exportar a un excel**



In [ ]:
def guardar_excel(bitacora,conteo,ruta_salida):
    
    #Crear el excel
    with pd.ExcelWriter(ruta_salida,engine='xlsxwriter') as writer:
        
        #Primera pagina con la bitacora
        bitacora.to_excel(writer,sheet_name='Bitacora',index=False)

        #Segunda pagina con el conteo por mes
        conteo.to_excel(writer,sheet_name='Conteos_por_mes', index=False)

    #Nombre del archivo de salida que se guarda en el escritorio de la maquina que ejecute el programa
    print(ruta_salida)

**Actualización de la base de datos**

In [ ]:
def actualizar_base_datos(bitacora,conn_teradata):

    with cnxn.connect() as conn:
        bitacora.to_sql('bitacora_eventos', conn, if exist='replace', index=False)

**Ejecución del código principal**

In [ ]:
def ejecutar_proceso(teradata_host_name, teradata_user, teradata_password, archivo_salida='bitacora.xlsx'):
    # Pedir al usuario que elija entre cargar un archivo CSV o desde Teradata
    print("Selecciona una opción:")
    print("1. Cargar desde archivo CSV")
    print("2. Cargar desde Teradata")
    
    opcion = input("Ingresa 1 para CSV o 2 para Teradata: ")

    # Validar la opción ingresada
    while opcion not in ['1', '2']:
        print("Opción no válida. Por favor ingresa 1 para CSV o 2 para Teradata.")
        opcion = input("Ingresa 1 para CSV o 2 para Teradata: ")

    # Cargar datos según la opción elegida
    if opcion == '1':
        # Pedir la ruta del archivo CSV
        ruta_archivo = input("Introduce la ruta del archivo CSV: ")
        while not os.path.isfile(ruta_archivo):
            print("La ruta no es válida o el archivo no existe. Intenta de nuevo.")
            ruta_archivo = input("Introduce la ruta del archivo CSV: ")
        # Cargar los datos del archivo CSV
        df = cargar_datos(ruta_archivo)
    elif opcion == '2':
        # Pedir el nombre de la tabla si se va a cargar desde Teradata
        nombre_tabla = input("Introduce el nombre de la tabla en Teradata: ")
        # Crear la conexión a Teradata
        conexion_teradata = create_engine(f'teradata://{teradata_user}:{teradata_password}@{teradata_host_name}/')
        # Cargar los datos desde Teradata
        df = cargar_datos_teradata(conexion_teradata, nombre_tabla)

    # Generar la bitácora
    bitacora = identificar_eventos(df)

    # Contar los eventos
    conteo_por_mes = contar_eventos(bitacora)

    # Exportar a un archivo Excel
    guardar_excel(bitacora, conteo_por_mes, archivo_salida)

    # Conectar a Teradata para actualizar la base de datos
    conexion_teradata = create_engine(f'teradata://{teradata_user}:{teradata_password}@{teradata_host_name}/')

    # Actualizar la base de datos en Teradata
    actualizar_base_datos(bitacora, conexion_teradata)
    
    # Mensaje de aprobación
    print("Proceso completado. La bitácora ha sido actualizada.")


**Uso del script**

La ruta del archivo de sebe ingresar como ...

Para ejecutar el proceso se deben considerar las credenciales definidas anteriormente, esta se reciben como argumento para la función

In [ ]:
if __name__ == "__main__":
    
    from sqlalchemy_teradata import teradata

    #Ejecutar el proceso con las credenciales definidas anteriormente    
    ejecutar_proceso(teradata_host_name,teradata_user,teradata_password, fuente_datos="teradata", nombre_table="")